# **Running Jupyter Code with Ollama Server**

1. **Download and Install Ollama**: Ensure that Ollama is downloaded and installed on your system.

2. **Check Ollama Version**:
   ```bash
   ollama --version
   ```

3. **Run Llama 3.2 Model**:
   To load the Llama 3.2 model:
   ```bash
   ollama run llama3.2
   ```

4. **Start Ollama Server**:
   To start the Ollama server, run:
   ```bash
   ollama serve
   ```

**Note**: If you face any issues while running the server, checking the background tasks using the port can help resolve conflicts. Always ensure the port is free before starting the server.

1. **Check if Ollama Server is Running**:
   Check if the server is running and using port 11434:
   ```bash
   netstat -ano | findstr :11434
   ```

2. **Identify and Kill Background Task (if needed)**:
   If you see that another task is using the port, you can identify and kill it:
   ```bash
   tasklist /FI "PID eq <PID>"
   taskkill /PID <PID> /F
   ```

3. **Close Ollama from Windows Task Bar (Optional)**:
   If the server is still not responding or you see issues, you can use the **up arrow key** (^) on the Windows task bar (bottom-left) to close any running Ollama instances.

4. **Start Jupyter Notebook**:
   Once the server is running, start your Jupyter notebook with the necessary code to interact with Ollama.

In [47]:
import ollama
from PyPDF2 import PdfReader
from IPython.display import display, Markdown ,update_display

In [49]:
system_prompt = '''You are an agent which analyses the resume and understands the context of resume and 
    extract the following fields and format the output:
    - Name
    - Email
    - Phone 1
    - Phone 2
    - Address
    - City
    - LinkedIn
    - Professional Experience (in years)
    - Highest Education
    - Is Fresher (yes/no)
    - Is Student (yes/no)
    - Skills (comma-separated)
    - Applied For Profile
    - Education (Institute Name, Year of Passing, Score)
    - Projects (Project Name, Description)
    - Professional Experience (Organisation Name, Duration, Profile)

    Ensure to format the output as:
    Name: [Value]
    Email: [Value]
    Phone 1: [Value]
    ... and provide this summary of resume in markdown format'''

In [50]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text
    
user_prompt = extract_text_from_pdf('./data/python-developer-resume-example.pdf') + "Provide a summary of this resume"

In [31]:
import requests
import json

def call_ollama(system_prompt, user_prompt):
    url = "http://localhost:11434/api/chat"
    
    payload = {
        "model": "llama3.2",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    }
    
    response = requests.post(url, json=payload)
    
    if response.status_code == 200:
        full_response = ""
        for line in response.iter_lines():
            if line:
                json_response = line.decode('utf-8')
                try:
                    response_data = json.loads(json_response)
                    if 'message' in response_data:
                        full_response += response_data['message'].get('content', '')
                except json.JSONDecodeError:
                    print("Error decoding JSON")
        
        return display(Markdown(full_response))
    else:
        print(f"Error: {response.status_code}")
        return None


response = call_ollama(system_prompt, user_prompt)
response

### Resume Summary

#### Contact Information
##### Name: Giulia González
##### Email: <gonzalez@email.com>
##### Phone 1: (123) 456-7890
##### Phone 2: N/A
##### Address: Detroit, MI
##### City: Detroit, MI
##### LinkedIn: N/A
##### Professional Profile Link: Github

#### Education

* **M.S. in Computer Science**, University of Chicago (2014 - 2016)
	+ Score: N/A
* **B.S. in Computer Science**, University of Pittsburgh (2010 - 2014)

#### Skills
HTML/CSS, SQL (PostgreSQL, Oracle), JavaScript (Angular), Python (Django), REST APIs (GraphQL), AWS (Redshift, S3), Git

#### Work Experience

*   **Python Developer**, DoorDash (September 2017 - Present)
    *   Duration: 6 years
    *   Profile:
        *   Built new Angular components for the customer-facing web app.
        *   Collaborated with an agile team to prioritize and scope feature requests.
        *   Reduced customer complaints by 23%.
        *   Boosted revenue by 6% through data pipelines discovery.
*   **Python Developer Intern**, Knewton (April 2016 - April 2017)
    *   Duration: 1 year
    *   Profile:
        *   Implemented RESTful APIs in Django for internal analytics team reporting speed increase.
        *   Reduced bugs reported by the client by 11% month-over-month.
        *   Provided project updates and design recommendations.

#### Projects

*   **Cryptocurrency Price Tracker**
    *   Creator
    *   Description:
        *   Incorporated API calls to several applications for efficient data storage in PostgreSQL backend.
        *   Utilized D3.js for user dynamic visualization of price movements over time.

## **OLD CODE**

**NOTE :** Just Kept the old code to show how Ollama provdes answers it sends one word at a time in a chunk type response 

In [32]:
import requests

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers)

# Print the raw response text for debugging
print("Raw Response:", response.text)

# Try to parse the response only if it's in JSON format
try:
    response_json = response.json()  # Try parsing the response as JSON
    print("Model Response:", response_json)
except ValueError as e:
    print("Error parsing JSON:", e)
    print("Response text may not be valid JSON.")


Raw Response: {"model":"llama3.2","created_at":"2024-12-18T10:37:04.3412715Z","message":{"role":"assistant","content":"The"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:04.5390128Z","message":{"role":"assistant","content":" capital"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:04.7603018Z","message":{"role":"assistant","content":" of"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:04.9268798Z","message":{"role":"assistant","content":" France"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:05.1936493Z","message":{"role":"assistant","content":" is"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:05.3531851Z","message":{"role":"assistant","content":" Paris"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:05.5912375Z","message":{"role":"assistant","content":"."},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T10:37:05.8094624Z","message":{"role":"assistant","conten

## **Easier Way**

In [51]:
import ollama

def call_ollama(system_prompt, user_prompt):
    stream = ollama.chat(model='llama3.2', messages=[
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': user_prompt
        }
    ])

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream['message']['content']:
        response += chunk or''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
        
# # Example usage
# system_prompt = "You are a helpful assistant who explains complex topics in simple terms."
# user_prompt = "What is 2+2"

response = call_ollama(system_prompt, user_prompt)
response

### Resume Summary
#### Contact Information
=====================================

*   **Name:** G I U L I A
*   **Email:** `ggonzalez@email.com`
*   **Phone 1:** `(123) 456-7890`
*   **LinkedIn:** Available
*   **Phone 2:** Not specified
*   **Address:** Detroit, MI

#### Professional Experience
==================================

*   **Python Developer**, DoorDash (September 2017 - Current)
    *   Duration: 5 years and 6 months
    *   Location: Detroit, MI
    *   Summary:
        
Python Developer
DoorDash
September 2017 - current Detroit, MI
·Worked on building new Angular components for the customer-facing web app, which improved the time on page for the average user by 2 minutes
·Collaborated with an agile team of 6, and helped prioritize and scope feature requests to ensure that the biggest impact features were worked on ﬁrst
·Built extensive test coverage for all new features, which reduced the number of customer complaints by 23%
·Acquired and ingested data to build and maintain data pipelines that led to discovering an opportunity for a new site feature, boosting revenue by 6%
·Communicated with internal teams and stakeholders, working to determine solutions for the user experience


*   **Python Developer Intern**, Knewton (April 2016 - April 2017)
    *   Duration: 1 year
    *   Location: Chicago, IL
    *   Summary:
        
Python Developer Intern
Knewton
April 2016 - April 2017 Chicago, IL
·Worked alongside another developer to implement RESTful APIs in Django that enabled internal analytics team to increase reporting speed by 24%
·Using Selenium, built out a unit testing infrastructure for a client web application that reduced the number of bugs reported by the client by 11% month over month
·Provided project updates to leadership team of 3, and offered recommendations for design
·Diagnosed issues causing slow speeds in applications, and documented the process to making the database query system more robust
·Participated in writing scalable code with a team of 4 interns and 1 developer for applications for a math course


#### Education
================

*   **M.S. Computer Science**, University of Chicago (2014 - 2016)
    *   Location: Chicago, IL
    *   Score: Not specified
*   **B.S. Computer Science**, University of Pittsburgh (2010 - 2014)
    *   Location: Pittsburgh, PA
    *   Score: Not specified

#### Skills
=============

*   HTML/ CSS
*   SQL (PostgreSQL, Oracle)
*   JavaScript (Angular)
*   Python (Django)
*   REST APIs (GraphQL)
*   AWS (Redshift, S3)
*   Git